## Probability Theory Pt.2: Compound Probability

this chapter deals with classic 'probability problems'

#### Puzzle problems

treasure ship:



1. Postulate three wrecked ships -- ship 1 with two gold chests (G-G), ship 2 with one gold one silver chest (G-S), and ship 3 with 2 silver chests (S-S)
2. Assert equal probabilitie of each ship being found
3. Step 2 implies equal probabilities for 6 chests to be found
4. Fact: Diver finds a chest of gold
    - we want to figure out the probability the other chest on the ship will be gold
5. Step 4 implies that ship 3 wasnt found, and can be removed
6. Three possibilities: 1) Diver found ship 1 chest a 2) Diver found ship 1 chest b 3) Diver found ship 2 chest 1
    - since step 2 all three possibilities have equal probability
7. If possibility is ship 1 chest a, then the other trunk is ship 1 chest b and vice versa; if possibility is ship 2 chest a, then the other chest is ship 2 chest b
8. From steps 6 and 7: each outcome has a probability of 1/3 since there is no other outcome
9. P(Gold) = P(ship 1 chest a) + P(ship 1 chest b) = 1/3 + 1/3 = 2/3
    - since the only way we get gold in the second chest is if we found ship one we can add up the probabilities of finding ship 1 chest a or b

Simulation

1. three urns containing the number (0,0) (0,1) and (1,1) where 0='gold' and 1='silver'
2. choose an urn at random
3. choose an element without replacement if 1 stop trial, if 0 continue
4. record the second element in the chosen urn
5. repeat steps 2-5 and calculate the proportion of 0s (should be 2/3)

In [37]:
import numpy as np
np.random.seed(123)

# create out ships
ships = np.array([[0,0], [0,1], [1,1]])

# randomly find a ship (many times)
selections = np.random.randint(0,3,100)

# randomly choose a chest on the ship, then the other chest
finds = np.array([np.random.choice(ships[selection], 2, replace=False) for selection in selections])

# remove all of the first found silver chests
gold_chests = np.delete(finds, finds[:,0] == 1, axis=0)

# find all the gold chests where the second chest is also gold
# then divide by the remaining gold chests to get proportion
sum(gold_chests[:,1] == 0) / len(gold_chests)

0.6382978723404256

We can see that we end up with 2/3 as we expected however I acknowledge this code is a little opaque. The tricky part of it is that we have 3 arrays and we need to draw a random element from that little array then take the other element. So my solution to this problem was to use a array of random numbers selected from {0,1,2} here I used 100 to work as my "ship finder" then I had to use `np.random.choice` on the selection from my ship finder. However there wasn't a good way to do this vectorized so I just put it into a list comprehension and converted that list back to an array. From here I deleted all the rows of the matrix where the first column was equal to 1 (where we found silver first). Then I only had a matrix where the first element was 0 so I just needed to count the rows where the second element was also 0 and divide that by the total remaining rows

Three door problem (monty hall problem)

The player faces three closed doors. One contans a prize and two are empty. After choosing a door one of the other doors is opend revealing that it is empty. The player is now given the opportunity of switching from their original choice to the other unopend door. What should the player do?

_Answer_: Switching doors doubles the chance of winning the prize

#### Examples of basic problems in probability

A poker problem: one pair (two of a kind)

What are the chances the first five cards chosen from the deck of 52 will contain two cards of the same denomination? We will only be using two of a kind (not 3 of a kind, 4 of a kind, or 2 pair).

In [43]:
# create a deck to sample from
deck = np.repeat(list(range(13)) ,[4]*13)
deck

array([ 0,  0,  0,  0,  1,  1,  1,  1,  2,  2,  2,  2,  3,  3,  3,  3,  4,
        4,  4,  4,  5,  5,  5,  5,  6,  6,  6,  6,  7,  7,  7,  7,  8,  8,
        8,  8,  9,  9,  9,  9, 10, 10, 10, 10, 11, 11, 11, 11, 12, 12, 12,
       12])

In [53]:
# create array of many decks to sample from
decks = np.array([deck]*1000)
print(f"shape of decks: {decks.shape}")
decks

shape of decks: (1000, 52)


array([[ 0,  0,  0, ..., 12, 12, 12],
       [ 0,  0,  0, ..., 12, 12, 12],
       [ 0,  0,  0, ..., 12, 12, 12],
       ...,
       [ 0,  0,  0, ..., 12, 12, 12],
       [ 0,  0,  0, ..., 12, 12, 12],
       [ 0,  0,  0, ..., 12, 12, 12]])

below we are going to use a functional apply to select a random sample of 5 cards from each deck without replacemnent

In [61]:
# now we're going to take a sample without replacement from each deck
def sample_no_replacement(deck):
    return np.random.choice(deck, 5, replace=False)

hands = np.apply_along_axis(sample_no_replacement, 1, decks)
print(f'shape of hands: {hands.shape}')
hands

shape of hands: (1000, 5)


array([[ 8, 11, 11,  3,  1],
       [10,  6,  5,  5,  0],
       [ 2,  7,  5,  4, 12],
       ...,
       [ 7,  2,  8,  0, 12],
       [ 7,  6,  0,  1,  5],
       [11,  0,  0,  3, 11]])

In [102]:
# figure out which hands are pairs and count them up
from collections import Counter

def is_pair(hand):
    if Counter(Counter(hand).values()) == {1:3, 2:1}:
        return True
    return False

sum(np.apply_along_axis(is_pair, 1, hands)) / 1000

0.426

This is a little convaluted. To determine if a hand had a pair in it, I checked to see if the counter which counts number of distinct values. However I have to do that again to the values of the first counter and make sure I count one 2 and three 1s. 3 different numbers and 1 pair produce a True from the function above and a False otherwise

Counter(1,2,3,4,4) -> {1:1, 2:1, 3:1, 4:2} -> Counter(1,1,1,2) -> {1:3, 2:1}

But we do end up getting very close to the right value that is 42%

~

Another intro poker problem

Which is more likely? two pairs or three of a kind

This is a comparison problem instead of an estimation problem.

We can use our draws from before and just estimate the frequencies of both from there. To identify the hand we need to first identify what the Counter(Counter()) frequencies would looklike

For two pair {2:2, 1:1} and for three of a kind its {3:1, 1:2}

In [112]:
def is_two_pair(hand):
    count = Counter(Counter(hand).values())
    if count == {2:2, 1:1}:
        return True
    return False

sum(np.apply_along_axis(is_two_pair, 1, hands)) / 1000

0.058

In [115]:
def is_three_kind(hand):
    count = Counter(Counter(hand).values())
    if count == {3:1, 1:2}:
        return True
    return False

sum(np.apply_along_axis(is_three_kind, 1, hands)) / 1000

0.023

#### the concepts of replacement and non-replacement

if we replace, conditions don't change

if we sample with replacement the sample draws remain independent of each other